# LAB 5 – Workflow Multiagente: Router + KB + Service Desk con Memoria

En este laboratorio construiremos un flujo multiagente completo para un **Service Desk interno**:

- `RouterAgent`:
  - Decide si la petición es informativa o requiere acción/ticket.
- `KnowledgeAgent`:
  - Consulta una **base de conocimiento local** (`kb/*.md`) para responder preguntas.
- `ServiceDeskAgentMemory`:
  - Gestiona tickets (crear / actualizar estado) en `data/tickets.csv`.
  - Recuerda preferencias del usuario en `data/profiles.json`.

El flujo será:

1. Router analiza la petición.
2. Si es de tipo “información” o conviene, se consulta primero la KB.
3. Si la KB no es suficiente o la petición es de “acción”, se pasa al Service Desk.


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

GITHUB_ENDPOINT = os.getenv("GITHUB_ENDPOINT")
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
GITHUB_MODEL = os.getenv("GITHUB_MODEL")

if not all([GITHUB_ENDPOINT, GITHUB_TOKEN, GITHUB_MODEL]):
    raise RuntimeError(
        "❌ Faltan variables en .env. Asegúrate de tener "
        "GITHUB_ENDPOINT, GITHUB_TOKEN y GITHUB_MODEL configuradas."
    )

# Adaptar a lo que espera OpenAIChatClient de agent-framework
os.environ["OPENAI_API_KEY"] = GITHUB_TOKEN
os.environ["OPENAI_BASE_URL"] = GITHUB_ENDPOINT
os.environ["OPENAI_CHAT_MODEL_ID"] = GITHUB_MODEL

print("✅ Configuración cargada desde .env")
print("  ENDPOINT:", GITHUB_ENDPOINT)
print("  MODEL:", GITHUB_MODEL)


✅ Configuración cargada desde .env
  ENDPOINT: https://models.github.ai/inference
  MODEL: gpt-4o


In [2]:
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient

import asyncio
import json
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Annotated

import pandas as pd
from pydantic import Field

# Carpetas de trabajo
DATA_DIR = Path("data")
OUT_DIR = Path("out")
KB_DIR = Path("kb")

DATA_DIR.mkdir(exist_ok=True)
OUT_DIR.mkdir(exist_ok=True)
KB_DIR.mkdir(exist_ok=True)

print("📂 DATA_DIR:", DATA_DIR.resolve())
print("📂 OUT_DIR:", OUT_DIR.resolve())
print("📂 KB_DIR:", KB_DIR.resolve())


📂 DATA_DIR: C:\Repos\agent-framework-workshop\data
📂 OUT_DIR: C:\Repos\agent-framework-workshop\out
📂 KB_DIR: C:\Repos\agent-framework-workshop\kb


In [3]:
TICKETS_CSV = DATA_DIR / "tickets.csv"

# Inicializar tickets.csv si no existe
if not TICKETS_CSV.exists():
    df_init = pd.DataFrame(
        columns=[
            "id", "fecha", "solicitante", "departamento",
            "categoria", "prioridad", "estado", "resumen", "detalle"
        ]
    )
    df_init.to_csv(TICKETS_CSV, index=False, encoding="utf-8")
    print(f"✅ Fichero de tickets creado: {TICKETS_CSV}")
else:
    print(f"✅ Fichero de tickets encontrado: {TICKETS_CSV}")

def load_tickets() -> pd.DataFrame:
    """
    Carga el CSV de tickets en un DataFrame de pandas.
    Si no existe o está vacío, devuelve un DataFrame con las columnas estándar.
    """
    if TICKETS_CSV.exists():
        return pd.read_csv(TICKETS_CSV, dtype=str).fillna("")
    return pd.DataFrame(
        columns=[
            "id", "fecha", "solicitante", "departamento",
            "categoria", "prioridad", "estado", "resumen", "detalle"
        ]
    )

print("🔎 Vista previa de tickets.csv:")
display(load_tickets().head())


✅ Fichero de tickets encontrado: data\tickets.csv
🔎 Vista previa de tickets.csv:


,id,fecha,solicitante,departamento,categoria,prioridad,estado,resumen,detalle
0,1,2025-01-10,antonio.soto@empresa.local,IT,nuevo_equipo,alta,pendiente,Solicitud portátil para teletrabajo,Necesito un portátil con al menos 16GB de RAM ...
1,2,2025-01-12,maria.fraga@empresa.local,RRHH,certificado,media,resuelto_auto,Certificado de empresa,Solicito un certificado de empresa para presen...
2,3,2025-01-15,carlos.gomez@empresa.local,IT,incidencia,alta,en_progreso,VPN no funciona,No puedo conectar a la VPN corporativa desde m...
3,4,2025-01-18,laura.fernandez@empresa.local,Facilities,mantenimiento,media,pendiente,Problema con aire acondicionado,En la sala de reuniones 3A el aire acondiciona...
4,5,2025-01-20,antonio.soto@empresa.local,RRHH,vacaciones,baja,resuelto_auto,Consulta sobre días de vacaciones,Quisiera confirmar cuántos días de vacaciones ...


In [4]:
PROFILES_JSON = DATA_DIR / "profiles.json"

# Inicializar profiles.json si no existe
if not PROFILES_JSON.exists():
    with PROFILES_JSON.open("w", encoding="utf-8") as f:
        json.dump({}, f, ensure_ascii=False, indent=2)
    print(f"✅ Fichero de perfiles creado: {PROFILES_JSON}")
else:
    print(f"✅ Fichero de perfiles encontrado: {PROFILES_JSON}")

def load_all_profiles() -> Dict[str, Dict]:
    if PROFILES_JSON.exists():
        with PROFILES_JSON.open("r", encoding="utf-8") as f:
            return json.load(f)
    return {}

def save_all_profiles(profiles: Dict[str, Dict]) -> None:
    with PROFILES_JSON.open("w", encoding="utf-8") as f:
        json.dump(profiles, f, ensure_ascii=False, indent=2)

def get_user_profile(email: str) -> Dict:
    profiles = load_all_profiles()
    email_key = email.lower()
    if email_key not in profiles:
        profiles[email_key] = {
            "email": email_key,
            "sede": "",
            "preferencias_equipo": "",
            "idioma_respuesta": "es"
        }
        save_all_profiles(profiles)
    return profiles[email_key]

def update_user_profile(email: str, sede: str = "", preferencias_equipo: str = "", idioma_respuesta: str = "") -> Dict:
    profiles = load_all_profiles()
    email_key = email.lower()
    perfil = profiles.get(email_key, {
        "email": email_key,
        "sede": "",
        "preferencias_equipo": "",
        "idioma_respuesta": "es"
    })

    if sede:
        perfil["sede"] = sede
    if preferencias_equipo:
        perfil["preferencias_equipo"] = preferencias_equipo
    if idioma_respuesta:
        perfil["idioma_respuesta"] = idioma_respuesta

    profiles[email_key] = perfil
    save_all_profiles(profiles)
    return perfil

# Prueba rápida
print("🧠 Perfil de ejemplo:", update_user_profile("antonio.soto@empresa.local", sede="Barcelona", preferencias_equipo="Lenovo"))


✅ Fichero de perfiles encontrado: data\profiles.json
🧠 Perfil de ejemplo: {'preferencias_equipo': 'Lenovo', 'sede': 'Barcelona'}


In [5]:
def crear_ticket_fc(
    desde_email: Annotated[str, Field(description="Dirección de correo del solicitante")],
    departamento: Annotated[str, Field(description="Departamento destino: IT, RRHH, Facilities u otro")],
    categoria: Annotated[str, Field(description="Categoria de la solicitud: nuevo_equipo, incidencia, vacaciones, certificado, mantenimiento, otro")],
    prioridad: Annotated[str, Field(description="Prioridad: alta, media o baja")],
    resumen: Annotated[str, Field(description="Resumen corto de la solicitud")],
    detalle: Annotated[str, Field(description="Detalle extendido de la solicitud")]
) -> Dict:
    """
    Crea un nuevo ticket en tickets.csv y guarda un JSON en out/ticket_{id}.json.
    Devuelve el diccionario con los datos completos del ticket creado.
    """
    data = load_tickets()

    if data.empty:
        next_id = 1
    else:
        ids = pd.to_numeric(data["id"], errors="coerce")
        next_id = int(ids.max() or 0) + 1

    nuevo_ticket = {
        "id": str(next_id),
        "fecha": datetime.now().strftime("%Y-%m-%d"),
        "solicitante": desde_email,
        "departamento": departamento or "Otro",
        "categoria": categoria or "otro",
        "prioridad": prioridad or "media",
        "estado": "pendiente",
        "resumen": (resumen or "")[:200],
        "detalle": detalle or "",
    }

    data = pd.concat([data, pd.DataFrame([nuevo_ticket])], ignore_index=True)
    data.to_csv(TICKETS_CSV, index=False, encoding="utf-8")

    ticket_json_path = OUT_DIR / f"ticket_{next_id}.json"
    with ticket_json_path.open("w", encoding="utf-8") as f:
        json.dump(nuevo_ticket, f, ensure_ascii=False, indent=2)

    print(f"✅ [Tool] Ticket creado con id={next_id}")
    return nuevo_ticket


def actualizar_estado_ticket_fc(
    id_ticket: Annotated[int, Field(description="Identificador numérico del ticket a actualizar")],
    nuevo_estado: Annotated[str, Field(description="Nuevo estado del ticket, por ejemplo: pendiente, en_progreso, resuelto_auto, resuelto_humano, cancelado")]
) -> Dict:
    """
    Actualiza el campo 'estado' de un ticket en tickets.csv y en el JSON correspondiente (si existe).
    Devuelve el ticket actualizado, o un mensaje de error si el id no existe.
    """
    data = load_tickets()
    if data.empty:
        return {
            "ok": False,
            "mensaje": f"No hay tickets en el sistema. No se encontró el ticket con id={id_ticket}."
        }

    mask = data["id"] == str(id_ticket)
    if not mask.any():
        return {
            "ok": False,
            "mensaje": f"No se encontró el ticket con id={id_ticket}."
        }

    data.loc[mask, "estado"] = nuevo_estado
    data.to_csv(TICKETS_CSV, index=False, encoding="utf-8")

    ticket_actualizado = data.loc[mask].iloc[0].to_dict()

    ticket_json_path = OUT_DIR / f"ticket_{id_ticket}.json"
    if ticket_json_path.exists():
        with ticket_json_path.open("w", encoding="utf-8") as f:
            json.dump(ticket_actualizado, f, ensure_ascii=False, indent=2)

    print(f"✅ [Tool] Estado del ticket {id_ticket} actualizado a '{nuevo_estado}'")
    return {
        "ok": True,
        "ticket": ticket_actualizado
    }


In [6]:
def actualizar_perfil_usuario_fc(
    email: Annotated[str, Field(description="Email corporativo del usuario para guardar su perfil")],
    sede: Annotated[str, Field(description="Ciudad o sede principal de trabajo del usuario")] = "",
    preferencias_equipo: Annotated[str, Field(description="Preferencia de marca o tipo de portátil (p.ej. Lenovo, Dell, etc.)")] = "",
    idioma_respuesta: Annotated[str, Field(description="Idioma preferido para las respuestas (es, en, etc.)")] = ""
) -> Dict:
    """
    Actualiza el perfil de usuario (sede, preferencias de equipo, idioma) en profiles.json.
    Devuelve el perfil completo después de la actualización.
    """
    perfil = update_user_profile(
        email=email,
        sede=sede,
        preferencias_equipo=preferencias_equipo,
        idioma_respuesta=idioma_respuesta
    )
    print(f"✅ [Tool] Perfil actualizado para {email}: {perfil}")
    return perfil


In [7]:
# Crear documentación de ejemplo si kb/ está vacío
if not any(KB_DIR.glob("*.md")):
    (KB_DIR / "politica_vacaciones.md").write_text(
        "# Política de Vacaciones\n\n- Días base: 23 laborables.\n- +1 día si antigüedad > 5 años.\n- +2 días si antigüedad > 10 años.\n",
        encoding="utf-8"
    )
    (KB_DIR / "politica_portatiles.md").write_text(
        "# Política de Portátiles\n\n- Un portátil por persona.\n- Segundo portátil solo en casos excepcionales aprobados por IT.\n",
        encoding="utf-8"
    )
    print("✅ KB de ejemplo creada en kb/")

print("📄 Ficheros en kb/:", [f.name for f in KB_DIR.glob('*.md')])

def kb_list_files() -> List[str]:
    """Devuelve la lista de .md en la KB."""
    return [f.name for f in KB_DIR.glob("*.md")]

def kb_read_file(
    filename: Annotated[str, Field(description="Nombre de fichero .md dentro de kb/")]
) -> str:
    """Lee el contenido de un fichero de la KB."""
    path = KB_DIR / filename
    if not path.exists():
        raise FileNotFoundError(f"No se encontró el fichero {filename} en kb/.")
    return path.read_text(encoding="utf-8")


📄 Ficheros en kb/: ['faq_it.md', 'faq_rrhh.md', 'politica_portatiles.md', 'politica_vacaciones.md']


In [ ]:
KNOWLEDGE_INSTRUCTIONS = """
Eres un agente especializado en consultar la base de conocimiento interna (KB).

Dispones de las funciones:
- kb_list_files(): para ver qué documentos hay en la KB.
- kb_read_file(filename): para leer el contenido de un documento .md en texto.

Tu trabajo:
1. Analizar la pregunta del usuario.
2. Decidir qué documentos de la KB pueden contener la respuesta.
3. Usar kb_read_file para leer solo los documentos relevantes.
4. Responder en español, citando únicamente información encontrada en la documentación.
5. Si la KB no tiene la respuesta, dilo claramente y sugiere abrir un ticket.

Reglas:
- No inventes políticas ni normas que no estén en los documentos.
- Sé concreto y menciona solo lo relevante.
"""

async def create_knowledge_agent() -> ChatAgent:
    chat_client = OpenAIChatClient()
    agent = chat_client.create_agent(
        name="KnowledgeAgent",
        instructions=KNOWLEDGE_INSTRUCTIONS,
        tools=[kb_list_files, kb_read_file],
    )
    return agent


In [ ]:

#➡️ Para representar de forma limpia la decisión del RouterAgent 
# (el agente que decide si la petición va a KB o a Service Desk) 
# sin usar clases manuales ni diccionarios desordenados.

@dataclass
class OrchestrationDecision:
    tipo: str               # "informacion" o "accion"
    usar_kb_primero: bool
    descripcion: str

    @staticmethod
    def from_json_str(json_str: str) -> "OrchestrationDecision":
        data = json.loads(json_str)
        return OrchestrationDecision(
            tipo=data.get("tipo", "informacion"),
            usar_kb_primero=bool(data.get("usar_kb_primero", True)),
            descripcion=data.get("descripcion", "")
        )

ROUTER_INSTRUCTIONS = """
Eres un agente router de un sistema de Service Desk multiagente.

Tu tarea es LEER la solicitud del usuario y devolver SIEMPRE un único objeto JSON con esta estructura:

{
  "tipo": "informacion" | "accion",
  "usar_kb_primero": true | false,
  "descripcion": "Explicación breve de la decisión"
}

Definiciones:
- "informacion": la petición se responde principalmente con información (políticas, FAQs).
- "accion": la petición implica crear o actualizar un ticket (solicitudes, incidencias, cambios de estado).
- "usar_kb_primero": true si tiene sentido intentar contestar primero con documentación, false si conviene ir directo al Service Desk.

Reglas:
- No expliques nada fuera del JSON.
- No añadas comentarios antes o después del JSON.
"""

async def create_router_agent() -> ChatAgent:
    chat_client = OpenAIChatClient()
    agent = chat_client.create_agent(
        name="RouterAgent",
        instructions=ROUTER_INSTRUCTIONS,
    )
    return agent

async def router_decide(texto_usuario: str) -> OrchestrationDecision:
    agent = await create_router_agent()
    thread = agent.get_new_thread()

    result = await agent.run(texto_usuario, thread=thread)
    raw_json = result.text.strip()
    print("🧭 RouterAgent JSON:\n", raw_json, "\n")

    decision = OrchestrationDecision.from_json_str(raw_json)
    print("🧭 Decisión interpretada:", decision)
    return decision


In [10]:
SERVICE_DESK_MEMORY_INSTRUCTIONS = """
Eres un agente de Service Desk interno con capacidad de recordar preferencias del usuario.

Tu trabajo es:
1. Entender la solicitud del usuario.
2. Usar el perfil del usuario (sede, preferencias de equipo, idioma) que se te pasa como JSON en el mensaje.
3. Decidir si se debe crear un ticket nuevo o actualizar uno existente.
4. Usar las funciones:
   - crear_ticket_fc
   - actualizar_estado_ticket_fc
   - actualizar_perfil_usuario_fc
5. Responder confirmando las acciones realizadas.

Reglas:
- Si el usuario da información de perfil (sede, marca preferida), llama a actualizar_perfil_usuario_fc.
- Pregunta por el email si no lo conoces antes de crear o actualizar tickets.
- No inventes IDs ni emails.
"""

async def create_service_desk_agent_with_memory() -> ChatAgent:
    chat_client = OpenAIChatClient()
    agent = chat_client.create_agent(
        name="ServiceDeskAgentMemory",
        instructions=SERVICE_DESK_MEMORY_INSTRUCTIONS,
        tools=[crear_ticket_fc, actualizar_estado_ticket_fc, actualizar_perfil_usuario_fc],
    )
    return agent

async def run_with_user_profile(agent: ChatAgent, email_usuario: str, texto_usuario: str, thread=None):
    """
    Inyecta el perfil del usuario como contexto y llama al agente.
    """
    perfil = get_user_profile(email_usuario)
    perfil_json = json.dumps(perfil, ensure_ascii=False)

    mensaje = (
        f"Perfil actual del usuario (JSON): {perfil_json}\n\n"
        f"Mensaje del usuario: {texto_usuario}"
    )

    if thread is None:
        thread = agent.get_new_thread()

    result = await agent.run(mensaje, thread=thread)
    return result, thread


In [11]:
async def get_knowledge_agent():
    """
    Devuelve un KnowledgeAgent y una función cleanup (por compatibilidad con versiones futuras MCP).
    Aquí no necesitamos cleanup, así que devolvemos un no-op.
    """
    agent = await create_knowledge_agent()
    async def cleanup():
        pass
    return agent, cleanup


In [12]:
async def run_service_workflow(email_usuario: str, texto_usuario: str) -> str:
    """
    Workflow multiagente:

    1. RouterAgent decide el tipo de petición y si usar KB primero.
    2. Si tipo == "informacion" o usar_kb_primero == True:
       - KnowledgeAgent intenta responder con documentación.
       - Si la respuesta parece suficiente, devolvemos esa respuesta.
    3. Si la KB no es suficiente o tipo == "accion":
       - Delegamos en ServiceDeskAgentMemory (tickets + memoria).
    """
    print("=== [Paso 1] RouterAgent ===")
    decision = await router_decide(texto_usuario)

    knowledge_agent, kb_cleanup = await get_knowledge_agent()
    service_agent = await create_service_desk_agent_with_memory()

    thread_kb = knowledge_agent.get_new_thread()
    thread_sd = service_agent.get_new_thread()

    try:
        if decision.tipo == "informacion" or decision.usar_kb_primero:
            print("=== [Paso 2] KnowledgeAgent (KB) ===")
            kb_result = await knowledge_agent.run(texto_usuario, thread=thread_kb)
            respuesta_kb = kb_result.text.strip()
            print("🤖 KnowledgeAgent:\n", respuesta_kb, "\n")

            lower = respuesta_kb.lower()
            kb_util = not (
                "no encuentro" in lower or
                "no aparece en" in lower or
                "no está en la base de conocimiento" in lower
            )

            if kb_util and decision.tipo == "informacion":
                print("✅ Workflow: la KB resuelve la petición de información.")
                return respuesta_kb

            print("ℹ️ Workflow: la KB no es suficiente, pasamos al Service Desk.")

        print("=== [Paso 3] ServiceDeskAgentMemory ===")
        sd_result, _ = await run_with_user_profile(service_agent, email_usuario, texto_usuario, thread_sd)
        respuesta_sd = sd_result.text.strip()
        print("🤖 ServiceDeskAgentMemory:\n", respuesta_sd, "\n")
        return respuesta_sd

    finally:
        await kb_cleanup()


In [13]:
async def demo_workflow_multiagente():
    email = "antonio.soto@empresa.local"

    casos = [
        "¿Cuántos días de vacaciones tengo según la política?",
        "Quiero pedir mis vacaciones del 1 al 15 de agosto.",
        "Quiero que recuerdes que trabajo en Barcelona y prefiero portátiles Lenovo.",
        "No puedo conectarme a la VPN y tengo una reunión urgente con un cliente."
    ]

    for texto in casos:
        print("=" * 80)
        print(f"👤 Usuario ({email}): {texto}\n")
        respuesta = await run_service_workflow(email, texto)
        print("💬 Respuesta final del workflow:\n", respuesta, "\n")

await demo_workflow_multiagente()


👤 Usuario (antonio.soto@empresa.local): ¿Cuántos días de vacaciones tengo según la política?

=== [Paso 1] RouterAgent ===
🧭 RouterAgent JSON:
 {
  "tipo": "informacion",
  "usar_kb_primero": true,
  "descripcion": "La solicitud se refiere a una política que probablemente está documentada en la base de conocimientos."
} 

🧭 Decisión interpretada: OrchestrationDecision(tipo='informacion', usar_kb_primero=True, descripcion='La solicitud se refiere a una política que probablemente está documentada en la base de conocimientos.')
=== [Paso 2] KnowledgeAgent (KB) ===
🤖 KnowledgeAgent:
 Según la política de vacaciones:

1. **Días laborables de vacaciones**:
   - La plantilla tiene **23 días laborables de vacaciones al año**.
   - Si tienes antigüedad superior a **5 años**, se te añade 1 día adicional (24 días).
   - Si tienes antigüedad superior a **10 años**, se te añaden 2 días adicionales (25 días).

Estos días son independientes de los festivos nacionales, autonómicos y locales. Para cual